In [1]:
import pymssql
import pathlib
import pydicom
import os
import pathlib
import numpy as np
import h5py
import xml.etree.ElementTree as ET
from scipy.io import savemat
import pandas as pd
import math
from tqdm import tqdm, trange


In [8]:
patientID = os.listdir('D:\LDCT_TEST')

In [39]:

print(Dir)

D:\LDCT_TEST\979011\CT0600290M\20151009\13240000


In [41]:
connect = pymssql.connect(
    host='172.31.6.157',
    user='TEST',
    password='81218',
    database='AICH'
)
PatientInfo_cursor = connect.cursor(as_dict=True)
LDCTInfo_cursor = connect.cursor(as_dict=True)
Annotation_cursor = connect.cursor(as_dict=True)

In [42]:
query_search_PatientInfo = '''
    select * from LDCT_PatientInfo where PartiID in (
        select distinct seriesDes from ExamStudySeries_5 as a
        inner join allEvents as b on a.eventID=b.eventID
        where chartNo=%(chartNo)s
    )
'''

In [43]:
query_search_LDCT_info = '''
    select * from LDCT_info as a
    inner join LDCT_Manufacturer as b on a.ManuID=b.ManuID
    where PartiID in (
        select distinct seriesDes from ExamStudySeries_5 as a
        inner join allEvents as b on a.eventID=b.eventID
        where chartNo=%(chartNo)s
    )
'''

In [44]:
query_search_annotation = '''
    select * from(
        select * from Localization where Disease=5
    ) as a
    inner join(
        select a.*,b.chartNo from ExamStudySeries_5 as a 
        inner join allEvents as b on a.eventID=b.eventID
    )as b 
    on a.PID=b.chartNo and a.StudyID=b.studyID and a.seriesID=b.seriesID and a.SD=b.studyDate
    where PID = %(chartNo)s
'''

In [69]:
for chartNo in patientID:
    for dirPath, dirNames, fileNames in os.walk(os.path.join('D:\LDCT_TEST',chartNo)):
        Dir=dirPath
    '''-----------------------------------------------------------------------'''
    PatientInfo_cursor.execute(query_search_PatientInfo, {'chartNo': chartNo})
    LDCTInfo_cursor.execute(query_search_LDCT_info, {'chartNo': chartNo})
    Annotation_cursor.execute(query_search_annotation, {'chartNo': chartNo})
    print(len([i for i,_ in enumerate(Annotation_cursor)]))
    if len([i for i,_ in enumerate(Annotation_cursor)])>0:
        '''-----------------------------------------------------------------------'''
        Main = ET.Element('Annotation')
        HospitalInfo = ET.SubElement(Main, 'HospitalInfo')
        Hospital = ET.SubElement(HospitalInfo, 'PatientID')
        Hospital.text = '06'
        '''-----------------------------------------------------------------------'''
        PatientInfo = ET.SubElement(Main, 'PatientInfo')
        PatientID = ET.SubElement(PatientInfo, 'PatientID')
        PatientID.text = 'Anonymous'
        ParticipantID = ET.SubElement(PatientInfo, 'ParticipantID')
        ParticipantID.text = str(PatientInfo_cursor.fetchone()['PartiID'])
        LDCTDate = ET.SubElement(PatientInfo, 'LDCTDate')
        LDCTDate.text = str(LDCTInfo_cursor.fetchone()['LDCTDate'])
        SeriesNumber = ET.SubElement(PatientInfo, 'SeriesNumber')
        SeriesNumber.text = str(LDCTInfo_cursor.fetchone()['SeriesNumber'])
        Age = ET.SubElement(PatientInfo, 'Age')
        Age.text = str(PatientInfo_cursor.fetchone()['Age'])
        Sex = ET.SubElement(PatientInfo, 'Sex')
        Sex.text = str(PatientInfo_cursor.fetchone()['Sex'])
        BirthDate = ET.SubElement(PatientInfo, 'BirthDate')
        BirthDate.text = str(PatientInfo_cursor.fetchone()['BirthDate'])
        Height = ET.SubElement(PatientInfo, 'Height')
        Height.text = str(PatientInfo_cursor.fetchone()['Height'])
        Weight = ET.SubElement(PatientInfo, 'Weight')
        Weight.text = str(PatientInfo_cursor.fetchone()['Weight'])
        '''-----------------------------------------------------------------------'''
        LDCTInfo = ET.SubElement(Main, 'LDCTInfo')
        TotalDLP = ET.SubElement(LDCTInfo, 'TotalDLP')
        TotalDLP.text = str(LDCTInfo_cursor.fetchone()['TotalDLP'])
        CTDIvol = ET.SubElement(LDCTInfo, 'CTDIvol')
        CTDIvol.text = str(LDCTInfo_cursor.fetchone()['CTDIvol'])
        InstitutionName = ET.SubElement(LDCTInfo, 'InstitutionName')
        InstitutionName.text = str(LDCTInfo_cursor.fetchone()['Institution'])
        ImageType = ET.SubElement(LDCTInfo, 'ImageType')
        ImageType.text = str(LDCTInfo_cursor.fetchone()['ImageType'])
        Manufacturer = ET.SubElement(LDCTInfo, 'Manufacturer')
        Manufacturer.text = str(LDCTInfo_cursor.fetchone()['ManuModel'])
        ScannerModel = ET.SubElement(LDCTInfo, 'ScannerModel')
        ScannerModel.text = str(LDCTInfo_cursor.fetchone()['Manufacturer'])
        DataCollectionDiameter = ET.SubElement(LDCTInfo, 'DataCollectionDiameter')
        DataCollectionDiameter.text = str(LDCTInfo_cursor.fetchone()['DataColDia'])
        ReconDiameter = ET.SubElement(LDCTInfo, 'ReconDiameter')
        ReconDiameter.text = str(LDCTInfo_cursor.fetchone()['ReconDia'])
        Exposure = ET.SubElement(LDCTInfo, 'Exposure')
        Exposure.text = str(LDCTInfo_cursor.fetchone()['Exposure'])
        XRayTubeCurrent = ET.SubElement(LDCTInfo, 'XRayTubeCurrent')
        XRayTubeCurrent.set('Max',str(LDCTInfo_cursor.fetchone()['MaxTubeCurr']))
        XRayTubeCurrent.set('Min',str(LDCTInfo_cursor.fetchone()['MinTubeCurr']))
        XRayTubeCurrent.set('Avg',str(LDCTInfo_cursor.fetchone()['AvgTubeCurr']))
        ExposuremAs = ET.SubElement(LDCTInfo, 'ExposuremAs')
        ExposuremAs.set('Total',str(LDCTInfo_cursor.fetchone()['TotalmAs']))
        ExposuremAs.set('Max',str(LDCTInfo_cursor.fetchone()['MaxmAs']))
        ExposuremAs.set('Min',str(LDCTInfo_cursor.fetchone()['MinmAs']))
        ExposuremAs.set('Avg',str(LDCTInfo_cursor.fetchone()['AvgmAs']))
        ConvolutionKernel = ET.SubElement(LDCTInfo, 'ConvolutionKernel')
        ConvolutionKernel.text = str(LDCTInfo_cursor.fetchone()['ConvKernel'])
        SingleCollimationWidth = ET.SubElement(LDCTInfo, 'SingleCollimationWidth')
        SingleCollimationWidth.text = str(LDCTInfo_cursor.fetchone()['SingleCollWid'])
        TotalCollimationWidth = ET.SubElement(LDCTInfo, 'TotalCollimationWidth')
        TotalCollimationWidth.text = str(LDCTInfo_cursor.fetchone()['TotalCollWid'])
        SeriesDescription = ET.SubElement(LDCTInfo, 'SeriesDescription')
        SeriesDescription.text = str(LDCTInfo_cursor.fetchone()['SeriesDesc'])
        SliceThickness = ET.SubElement(LDCTInfo, 'SliceThickness')
        SliceThickness.text = str(LDCTInfo_cursor.fetchone()['SliceThickness'])
        SpacingBtSlices = ET.SubElement(LDCTInfo, 'SpacingBtSlices')
        SpacingBtSlices.text = str(LDCTInfo_cursor.fetchone()['SpacingBtSlices'])
        Gap = ET.SubElement(LDCTInfo, 'Gap')
        Gap.text = str(LDCTInfo_cursor.fetchone()['Gap'])
        KVP = ET.SubElement(LDCTInfo, 'KVP')
        KVP.text = str(LDCTInfo_cursor.fetchone()['KVP'])
        Pitch = ET.SubElement(LDCTInfo, 'Pitch')
        Pitch.text = str(LDCTInfo_cursor.fetchone()['Pitch'])
        res = Annotation_cursor.fetchall()
        for row in res:
                if row['LabelName']=='OtherFinding':
                        OtherFinding = ET.SubElement(LDCTInfo, 'OtherFinding')
                        OtherFinding.set('X_Pos',str(row['x']))
                        OtherFinding.set('Y_Pos',str(row['y']))
                        OtherFinding.set('Z_Pos',str(row['z']))
                        OtherFinding.set('ROISize','50')
                else:
                        Abnormality = ET.SubElement(LDCTInfo, 'Abnormality')
                        Abnormality.set('X_Pos',str(row['x']))
                        Abnormality.set('Y_Pos',str(row['y']))
                        Abnormality.set('Z_Pos',str(row['z']))
                        Abnormality.set('ROISize','50')
        trees = ET.ElementTree(Main)
        filename = 'Annotation.xml'
        print(os.path.join(Dir,filename))
        #trees.write() 

TypeError: 'NoneType' object is not subscriptable

In [ ]:


root=r'\\172.31.6.6\share1\NFS\image'


'''
First, searching PatientID 
'''
list_file_dir = input('請輸入清單位置:')
user=input('請輸入所標註帳號:')
diseaseNo = input('請輸入研究主題編號:')
saveFilePath = input('請輸入儲存影像路徑:')
date1 = input('請輸入標註日期範圍(前)，格式:yyyy-mm-dd:')
date2 = input('請輸入標註日期範圍(後)，格式:yyyy-mm-dd:')
list_file = list(pathlib.Path(list_file_dir).glob("*.csv"))[0]
PatientListID=pd.read_csv(list_file,header=None).iloc[:,0].values.tolist()


cursor = connect.cursor(as_dict=True)
for i in tqdm(range(len(PatientListID))):
    #取得標記資料
    query = '''
    select * from Localization where PID=%(var1)s and username=%(var2)s and Disease=%(var3)s 
     and CONVERT(date,[date]) >= %(var4)s　and　CONVERT(date,[date]) <= %(var5)s
    '''
    cursor.execute(query, {'var1': PatientListID[i],'var2':user,'var3':diseaseNo,'var4':date1,'var5':date2})

    SD = []


    for row in cursor:
        SD.append(row['StudyID'])

    if len(SD)!=0:#代表有標記資料


        query = '''
        select PID ,SD,StudyID,seriesID from Localization where PID=%(var1)s and username=%(var2)s and Disease=%(var3)s and CONVERT(date,[date]) >= %(var4)s　and　CONVERT(date,[date]) <= %(var5)s group by PID,SD,StudyID,seriesID''' #取得影像位置資料
        cursor.execute(query, {'var1': PatientListID[i],'var2':user,'var3':diseaseNo,'var4':date1,'var5':date2})
        res = cursor.fetchall()

        for j in range(len(res)):
            

            PID = str(res[j]['PID'])
            MedExecTime = str(res[j]['SD'].replace('-',''))
            StudyID = str(res[j]['StudyID'])
            seriesID = str(res[j]['seriesID'])
            fileDir = os.path.join(r'\\172.31.6.6\share1\NFS\image',PID,MedExecTime,StudyID,seriesID)
            fileDir = fileDir.replace('-','')
            fileDir = fileDir.replace(' ', '')

            saveFiledir = os.path.join(saveFilePath,'image',PID,MedExecTime,StudyID,seriesID)
            pathlib.Path(saveFiledir).mkdir(parents=True, exist_ok=True)

            tempPath = list(pathlib.Path(fileDir).glob('*PETCT.h5'))

            paths = sorted([os.path.join(filename) for filename in tempPath])[0]
            with h5py.File(paths, "r") as f:
                a_group_key = list(f.keys())
                CT_header = f['CT_header']
                PET_header = f['PET_header']
                CT_tag = []
                PET_tag = []
                for header in CT_header:
                    CT_tag.append(pydicom.dataset.Dataset.from_json(header))      
                for header in PET_header:
                    PET_tag.append(pydicom.dataset.Dataset.from_json(header))         
                PET = np.array(f['PET_vol'])


            tempPath = list(pathlib.Path(fileDir).glob('*PETCTmod_Axial.h5'))
            paths = sorted([os.path.join(filename) for filename in tempPath])[0]
            with h5py.File(paths, "r") as f:
                CT_view = np.array(f['PETCT_vol'])[:, 1, :, :] #0是PET 1是CT_mod


            query = '''
            select * from Localization where PID=%(var1)s and username=%(var2)s and Disease=%(var3)s and StudyID=%(var4)s and seriesID=%(var5)s
            '''
            cursor.execute(query, {'var1': PID,'var2':user,'var3':diseaseNo,'var4':StudyID,'var5':seriesID})
            data = cursor.fetchall()
            #print('PID:',PID,' user:',user,' diseaseNo:',diseaseNo,' StudyID:',StudyID,' seriesID:',seriesID)
            Main = ET.Element('Main')
            Image_Info = ET.SubElement(Main, 'Image_Info')
            PatientID = ET.SubElement(Image_Info, 'PatientID')
            PatientID.text = str(PET_tag[0].PatientID)
            SeriesDescription = ET.SubElement(Image_Info, 'SeriesDescription')
            SeriesDescription.text = str(PET_tag[0].SeriesDescription)
            SeriesNumber = ET.SubElement(Image_Info, 'SeriesNumber')
            SeriesNumber.text =str(PET_tag[0].SeriesNumber)
            SeriesDate = ET.SubElement(Image_Info, 'SeriesDate')
            SeriesDate.text =str(PET_tag[0].ContentDate)
            SeriesTime = ET.SubElement(Image_Info, 'SeriesTime')
            SeriesTime.text =str(PET_tag[0].SeriesTime)
            SliceThickness = ET.SubElement(Image_Info, 'SliceThickness')
            SliceThickness.text =str(PET_tag[0].SliceThickness)
            Pixel_Spacing = ET.SubElement(Image_Info, 'Pixel_Spacing')
            Pixel_Spacing.text =str(PET_tag[0].PixelSpacing[0])
            CT_Pixel_Spacing = ET.SubElement(Image_Info, 'CT_Pixel_Spacing')
            CT_Pixel_Spacing.text =str(CT_tag[0].PixelSpacing[0])
            for ind, row in enumerate(data):
                Tumor_Area = ET.SubElement(Main, 'Tumor_Area')
                Tumor_Area.set('X', str(row['Click_X']))
                Tumor_Area.set('Y', str(row['Click_Y']))
                Tumor_Area.set('Z', str(row['Click_Z']))
                Tumor_Area.set('Area_size', '7cm')
                Tumor_Area.set('UserName', str(row['username']))
                Tumor_Center = ET.SubElement(Tumor_Area, 'Tumor_Center')
                Tumor_Center.set('C_Type', 'LocalMax')
                Tumor_Center.set('UserName', str(row['username']))
                Tumor_Center.set('memo', str(row['LabelName'][0:5]).replace(' ','_')+'_'+str(row['LabelRecord']).replace(' ','_')+'_'+str(ind))
                Tumor_Center.set('X', str(math.floor(float(row['x']))))
                Tumor_Center.set('Y', str(math.floor(float(row['y']))))
                Tumor_Center.set('Z', str(math.floor(float(row['z']))))
                Tumor_Center.text = str(row['SUV'])
            trees = ET.ElementTree(Main)
            filename = str(PET_tag[0].PatientID)+'_'+str(PET_tag[0].ContentDate)+'_'+str(PET_tag[0].SeriesNumber)+'_SOPT_log.xml'
            trees.write(os.path.join(saveFiledir,filename))

            filename = str(PET_tag[0].PatientID) + '_' + str(PET_tag[0].ContentDate) + '_' + str(
                PET_tag[0].SeriesNumber) + '_CT_Body.mat'
            newCT = CT_view.copy()
            newCT = newCT.transpose(1,2,0)
            CT_mat={'Data':np.float64(newCT)}
            savemat(os.path.join(saveFiledir,filename),CT_mat,do_compression=True)
            filename = str(PET_tag[0].PatientID) + '_' + str(PET_tag[0].ContentDate) + '_' + str(
                PET_tag[0].SeriesNumber) + '_PET_Body.mat'
            newPET=PET.copy()
            newPET = newPET.transpose(1,2,0)
            PET_mat={'Data':np.float64(newPET)}
            savemat(os.path.join(saveFiledir,filename),PET_mat,do_compression=True)

